# Подключение Google Drive с датасетами

Нужно для того, чтобы подтянуть датасет и ключи для SSH и WandB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Клонирование репозитория

# Установка зависимостей

В конце процесса может выдаваться пара предупреждений, но они касаются только тех пакетов, с которыми мы не будем работать:

```
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
```

In [ ]:
!pip install -q -U -r Gun-Detection/requirements.txt
!pip install -U pytorch-lightning==2.0.0

# Распаковка датасета

Датасет желательно разместить на локальной машине для того, чтобы ускорить доступ к нему через DataLoader, поэтому мы извлекаем данные в нужное место

In [6]:
!rm -rf dataset
!mkdir dataset
!7z x -o/content/dataset/ -y -bsp2 "/content/drive/MyDrive/datasets/"

"7z" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


# Добавление папки с кодом в python sys path

Нужно для того, чтобы иметь возможность импортировать модули напрямую из проекта

In [ ]:
import sys
sys.path.append('')

# Импортирование Transforms

**НУЖНО ДЕЙСТВИЕ**

Здесь вам необходимо добавить аугментаций к базовому набору аугментаций. Для этого изучите пакет albumentations и посмотрите, какие аугментации вы считаете полезными для задачи детекции.

Совершенно точно как минимум стоит использовать аугментацию RandomSizeCrop вместо CenterCrop, но возможно, что вы придумаете свой набор аугментаций для этой задачи.

При подборе аугментаций стоит учитывать, что некоторые из них работают очень быстро, другие -- достаточно долго. Поэтому добавляйте аугментации с умом.

Аугментации добавляйте в отдельный файл для того, чтобы не поломать выполняемость юнит-тестов. Например, в `dataset.transforms_custom.py`.



In [11]:
# You may need to change this import
# from dataset.transforms_basic import transforms
from dataset.transforms_custom import transforms

# Создание датасетов

Тут создаются датасеты. Если вы планируете расширять датасет, который будете использовать, стоит убедиться, что используется нужный путь до файлов при создании датасетов.

Если вы хотите использовать модифицированную версию датасета, сохраняйте ее в отдельный файл, чтобы не сломать выполнение юнит-тестов. Например, в `dataset/dataset_custom.py`

In [9]:
# You may need to change this import
from dataset.dataset import build_datasets

train_ds, test_ds = build_datasets(
    # '/content/dataset',
    '/content/dataset',
    transforms=transforms)

print(len(train_ds))
print(len(test_ds))

12103
1344


# Подготовка DataLoaders

Подготавливаются DataLoaders. Здесь нужно убедиться в том, что включена рандомизация на тренировочных данных, включена параллельная обработка семплов (количество выделенных потоков больше 0).

Также стоит убедиться, что на тренировочных данных последний батч отбрасывается.

In [10]:
from dataset.dataloaders import build_dataloaders

dl_train, dl_test = build_dataloaders(
    train_ds, test_ds,
    batch_size=32,
    shuffle=True,
    num_workers=8)

# Создание нейронной сети

Если вы захотите использовать какую-то свою модель, создайте отдельный файл для этой модели для того, чтобы не испортить исполнения юнит-тестов.

Например, вы можете использовать файл `models/model_custom.py`

In [12]:
# You may need to change this import
from models.model import RetinaRehead

model = RetinaRehead()

# Подготовка пайплайна

**Нужно действие**

Здесь готовится пайплайн обучения. В целом, мы запрограммировали процесс обучения в последней задаче. Однако стоит учесть, что можно добавить разнообразных трюков обучения как в процесс обучения, так и в `trainer` в виде `callbacks`.

Точно нужно будет сохранять веса нейронной сети на диск, для этого рекоммендуется использовать ModelCheckpoint.

Изучите, какие еще есть трюки обучения модели (Callbacks) и добавьте их в процесс обучения.

Если вы решите расширить код обучения из задания, сохраните новый код в новый файл, чтобы не испортить результат выполнения юнит-тестов. Например, вы можете использовать файл `lightning/lightning_custom.py`. В этом случае измените соответствующий импорт в коде ниже.

Обратите внимание, что вы можете изменять название проекта и название эксперимента.

Посмотреть процесс обучения можно на сайте [Weights and Biases](https://wandb.ai/).

In [18]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar

# You may need to change this import
from lightning.lightning import PLModel, build_trainer


model = RetinaRehead()
pl_model = PLModel(model)

trainer = build_trainer(
    # The parameters of trainer can be modified
    callbacks=[
        ModelCheckpoint(
            dirpath='./drive/MyDrive/checkpoints',
            filename='detector/{epoch}-{mAP-valid:.2f}',
            monitor='loss-valid',
            save_top_k=3,
            save_last=True
        ),
        TQDMProgressBar(refresh_rate=1)
    ],
    max_epochs=100,
    project='car-detection',
    name='retina',
    accelerator='auto',
    wandb_key='/content/drive/MyDrive/wandb/key.txt'
)

Seed set to 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\konto\.netrc
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


# Запуск обучения

In [19]:
trainer.fit(
    pl_model,
    dl_train,
    val_dataloaders=dl_test)

wandb: Currently logged in as: sweetb0nes (sweetb0ne5). Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                 | Params
---------------------------------------------------
0 | model     | RetinaRehead         | 38.2 M
1 | MAP_train | MeanAveragePrecision | 0     
2 | MAP_valid | MeanAveragePrecision | 0     
---------------------------------------------------
38.0 M    Trainable params
225 K     Non-trainable params
38.2 M    Total params
152.806   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\konto\anaconda3\envs\PythonGPU\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
c:\Users\konto\anaconda3\envs\PythonGPU\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
c:\Users\konto\anaconda3\envs\PythonGPU\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections

Training: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 4.00 GiB total capacity; 3.37 GiB already allocated; 0 bytes free; 3.45 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Цель обучения модели и способы ее улучшить
Цель обучения модели -- максимизировать метрику mAP50 на валидации и на тестировании для того, чтобы как можно больше автомобилей и номеров на изображении было найдено.

Максимизировать метрику можно различными способами. Вот три основных направления:

1. Работа над датасетом
2. Работа над моделью
3. Работа над лосс-функцией

Когда мы работаем над датасетом, мы стараемся либо добавить в него новых данных (что обычно дорого и трудозатратно), либо стараемся как-то переиспользовать уже имеющиеся данные. Мы посмотрим, каким образом можно расширять имеющиеся данные в следующей лекции.

Также мы можем использовать другие модели. Например, мы можем взять VisualTransformer или модель, предобученную на большем объеме данных. Либо мы можем найти какой-то совсем другой подход к задаче детекции.

Если мы понимаем, что используемая нами лосс-функция не оптимальна, необходимо решить эту проблему. Нам необходимо в этом случае продумать математику имеющейся проблемы и предложить ее решение.